In [1]:
import pandas as pd
df_tweets = pd.read_csv('dataset.csv')
df_tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


W pierwszej kolejności odczytywane są dane z pliku .csv. Przechowywane będą one w typie Pandas DataFrame.

In [2]:
df_tweets = df_tweets.drop(df_tweets[df_tweets['airline_sentiment_confidence'] < 0.5].index, axis = 0)

W celu zwiększenia prawdopodobieństwa prawidłowego określenia sentymentu wykluczone zostają wszystkie tweety w których prawdopodobieństwo poprawnej klasyfikacji wynosi mniej niż 50%.

In [8]:
X = df_tweets['text']
Y = df_tweets['airline_sentiment']

Ze względu na to, że zbiór zawiera wiele informacji, które nie będą potrzebne do wykonania projektu wybrane zostają tylko stosowne kategorie: treść tweeta oraz sentyment.

In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
stop_words = stopwords.words('english')
stemmer = PorterStemmer()

import re
df_tweets_clean=[]
for i in range(len(X)):
    tweet=re.sub('[^a-zA-Z]',' ',X.iloc[i])
    tweet = tweet.lower().split()
    tweet = [stemmer.stem(word) for word in tweet if (word not in stop_words)]
    tweet = ' '.join(tweet)
    df_tweets_clean.append(tweet)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Za pomocą wyrażeń regularnych oraz stemmera i listy słów stop dla języka angielskiego z `nltk` tekst tweetów przygotowywane zostaje do przetworzenia. Usunięte zostają wszelkie znaki poza literami, słowa stop oraz końcówki wyrazów.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3000, stop_words = ['virginamerica', 'unit'])
X_BoW = cv.fit_transform(df_tweets_clean).toarray()

Aby przetworzyć dane tekstowe na liczby użyta zostanie metoda "worka słów" ("bag of words"). Kategoryzuje ona częstotliwość występowania konkretnych słów w tweecie. Aby przygotować tekst w tym formacie użyty zostaje `CountVectorizer` z `sklearn`. Ze względu na tematykę zbioru danych (amerykańskie linie lotnicze) frazy `virginamerica` oraz `unit` (odpowiednio Virgin America Airlines oraz United Airlines) zostają potraktowane jako słowa stop - ich użycie w tych tweetach nie ma wpływu na sentyment.

In [17]:
sentiment_ordering = ['negative', 'neutral', 'positive']
Y = Y.apply(lambda x: sentiment_ordering.index(x))

Ze względu na to, że dane informacje o sentymencie zawarte zostały w zbiorze danych jako tekst konieczne będzie ich przekodowanie do postaci cyfr. 

In [18]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

`sklearn` oferuje gotowy model Naïve Bayes, który zostanie wykorzystany do analizy sentymentu tweetów.

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_BoW, Y, test_size = 0.3)
model.fit(X_train, Y_train)

MultinomialNB()

Funkcje dostępne w `sklearn` pozwalają na automatyczne podzielenie zbioru danych na zbiór treningowy i zbiór testowy. `test_size = 0.3` oznacza, że 70% zbioru danych zostanie przydzielone na zbiór treningowy, a 30% na zbiór testowy.

In [20]:
Y_pred=model.predict(X_test)
from sklearn.metrics import classification_report
cf = classification_report(Y_test, Y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.83      0.88      0.85      2742
           1       0.59      0.50      0.54       914
           2       0.69      0.68      0.69       666

    accuracy                           0.77      4322
   macro avg       0.70      0.69      0.69      4322
weighted avg       0.76      0.77      0.76      4322



`sklearn` udostępnia również możliwość wygenerowania raportu dokonanej klasyfikacji. Jak widać za pomocą Naïve Bayes udało się osiągnąć dokładność 77% względem przewidywanych odpowiedzi, co biorąc pod uwagę prostotę i szybkość projektu uznać można za wynik zadowalający. Jednym z działań jakie można byłoby podjąć w celu poprawy dokładności przewidywania wyników byłoby pozyskanie bardziej zrównoważonego zbioru danych - jak widać 2742 rozpatrywanych tweetów miało wartość negatywną, a jedynie 914 i 666 odpowiednio neutralną i pozytywną.

Źródło danych: [Twitter US Airline Sentiment at Kaggle.com](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment?resource=download)

Opracowane na podstawie: [Twitter Sentiment Analysis Using Python for Complete Beginners](https://medium.com/swlh/tweet-sentiment-analysis-using-python-for-complete-beginners-4aeb4456040)